In [9]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


DAC_SR = 4e9          
buf_len = 2**14         
dt = 1.0 / DAC_SR
X_axis = (1.0 / DAC_SR) * np.arange(0, buf_len)
freq = np.fft.fftshift(np.fft.fftfreq(buf_len, d=dt)) 

E0 = 1.0
G0_t = E0 * np.ones(buf_len)

line_freqs = np.array([0.0, 840e6, -1330e6])   # Hz
power_ratios = np.array([5.0, 3.0, 1.0])       # corresponding to carrier:+840:-1330 as 5:3:1
field_amps = np.sqrt(power_ratios)   # if ratios are power. If ratios already amplitude, skip sqrt.


sigma_hz = 1.0e6   # 1 MHz width (tune as needed)
target_amp = np.zeros_like(freq)

for f0, A in zip(line_freqs, field_amps):
    target_amp += A * np.exp(-0.5 * ((freq - f0) / sigma_hz)**2)

initial_field_w = np.fft.fftshift(np.fft.fft(np.fft.ifftshift(G0_t)))
norm_time_energy = np.linalg.norm(initial_field_w)   # L2 in freq domain proportional to time energy
norm_target = np.linalg.norm(target_amp)
if norm_target == 0:
    raise ValueError("target amplitude is zero (check line frequencies and sigma).")
target_amp *= (norm_time_energy / norm_target)

# -----------------------
# GSA iterative loop
# -----------------------
phi_t = np.random.uniform(0, 2*np.pi, buf_len)  # initial random phase
max_iters = 1000
tol = 1e-6

for it in range(max_iters):
    # forward: time -> freq
    field_t = G0_t * np.exp(1j * phi_t)
    field_w = np.fft.fftshift(np.fft.fft(np.fft.ifftshift(field_t)))
    amp_w = np.abs(field_w)
    phase_w = np.angle(field_w)

    # impose frequency amplitude constraint
    new_field_w = target_amp * np.exp(1j * phase_w)

    # inverse: freq -> time
    new_field_t = np.fft.fftshift(np.fft.ifft(np.fft.ifftshift(new_field_w)))
    amp_t = np.abs(new_field_t)
    phi_t = np.angle(new_field_t)

    # impose time amplitude constraint
    # set amplitude back to known G0_t, keep phase phi_t
    field_t_next = G0_t * np.exp(1j * phi_t)

    # convergence metric: compare current frequency amplitude to target
    err = np.linalg.norm(amp_w - target_amp) / np.linalg.norm(target_amp)
    if (it % 50 == 0) or (err < 1e-4):
        print(f"iter {it:4d}  rel_err={err:.3e}")
    if err < tol:
        print(f"Converged at iter {it}, rel_err={err:.3e}")
        break
else:
    print(f"Finished {max_iters} iters; final rel_err={err:.3e}")

# final recovered objects
final_phi_t = phi_t.copy()
final_field_t = G0_t * np.exp(1j * final_phi_t)
final_field_w = np.fft.fftshift(np.fft.fft(np.fft.ifftshift(final_field_t)))

# -----------------------
# Plotly figures
# -----------------------
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=(
                        "Time amplitude G0(t)",
                        "Frequency amplitude (target vs achieved)",
                        "Recovered phase φ(t)",
                        "Frequency-domain phase of final field"
                    ))

# (1,1) Time amplitude G0(t)
fig.add_trace(go.Scatter(x=t * 1e9, y=G0_t, name="G0(t)"), row=1, col=1)
fig.update_xaxes(title_text="Time (ns)", row=1, col=1)
fig.update_yaxes(title_text="Amplitude", row=1, col=1)

# (1,2) Frequency amplitude target vs achieved
fig.add_trace(go.Scatter(x=freq / 1e9, y=target_amp, name="target amp"), row=1, col=2)
fig.add_trace(go.Scatter(x=freq / 1e9, y=np.abs(final_field_w), name="achieved amp",
                          line=dict(dash='dash')), row=1, col=2)
fig.update_xaxes(title_text="Frequency (GHz)", range=[-3, 3], row=1, col=2)
fig.update_yaxes(title_text="Amplitude", row=1, col=2)

# (2,1) Recovered phase φ(t)
fig.add_trace(go.Scatter(x=t * 1e9, y=final_phi_t, name="φ(t)"), row=2, col=1)
fig.update_xaxes(title_text="Time (ns)", row=2, col=1)
fig.update_yaxes(title_text="Phase (rad)", row=2, col=1)

# (2,2) Frequency-domain phase of final field
fig.add_trace(go.Scatter(x=freq / 1e9, y=np.angle(final_field_w), name="phase"), row=2, col=2)
fig.update_xaxes(title_text="Frequency (GHz)", row=2, col=2)
fig.update_yaxes(title_text="Phase (rad)", row=2, col=2)

fig.update_layout(height=800, width=1100, title_text="Gerchberg–Saxton Results")
fig.show()

# -----------------------
# EOM drive waveform (what to send to the modulator)
# -----------------------
# The phase drive is proportional to φ(t). Without Vπ, show a zero-mean, unit-max waveform.
eom_drive = final_phi_t - np.mean(final_phi_t)
maxabs = np.max(np.abs(eom_drive))
eom_drive_norm = eom_drive / maxabs if maxabs > 0 else eom_drive

fig_eom = go.Figure()
fig_eom.add_trace(go.Scatter(x=t * 1e9, y=eom_drive_norm, name="Normalized EOM drive"))
fig_eom.update_layout(title="Final EOM Drive Waveform (normalized)",
                      xaxis_title="Time (ns)",
                      yaxis_title="Normalized voltage (arb.)")
fig_eom.show()

iter    0  rel_err=1.383e+00
iter   50  rel_err=2.152e-01
iter   50  rel_err=2.152e-01
iter  100  rel_err=1.977e-01
iter  100  rel_err=1.977e-01
iter  150  rel_err=1.974e-01
iter  150  rel_err=1.974e-01
iter  200  rel_err=1.974e-01
iter  200  rel_err=1.974e-01
iter  250  rel_err=1.974e-01
iter  250  rel_err=1.974e-01
iter  300  rel_err=1.974e-01
iter  300  rel_err=1.974e-01
iter  350  rel_err=1.974e-01
iter  350  rel_err=1.974e-01
iter  400  rel_err=1.974e-01
iter  400  rel_err=1.974e-01
iter  450  rel_err=1.974e-01
iter  450  rel_err=1.974e-01
iter  500  rel_err=1.974e-01
iter  500  rel_err=1.974e-01
iter  550  rel_err=1.974e-01
iter  550  rel_err=1.974e-01
iter  600  rel_err=1.974e-01
iter  600  rel_err=1.974e-01
iter  650  rel_err=1.974e-01
iter  650  rel_err=1.974e-01
iter  700  rel_err=1.974e-01
iter  700  rel_err=1.974e-01
iter  750  rel_err=1.974e-01
iter  750  rel_err=1.974e-01
iter  800  rel_err=1.974e-01
iter  800  rel_err=1.974e-01
iter  850  rel_err=1.974e-01
iter  850  rel